# The Neuropeptide *pth2* Dynamically Senses Others via Mechanosensation
Lukas Anneser$^{1}$, Ivan C. Alcantara$^{1,2}$, Anja Gemmer$^{1}$, Kristina Mirkes$^{1}$, Soojin Ryu$^{3}$, and Erin M. Schuman$^{1}$

1: Max Planck Institute for Brain Research, Frankfurt, Germany\
2: current address: Brown University, Providence, USA\
3: Living Systems Institute & College of Medicine and Health, University of Exeter, Exeter, UK

## Abstract
Species that depend on membership in social groups for survival exhibit changes in neuronal gene expression and behavior when they face restricted social interactions or isolation. Here we show that, across its lifespan, social isolation specifically decreased the transcription of the vertebrate-specific neuropeptide *pth2* in zebrafish, Danio rerio. Just 30 minutes of exposure to conspecifics, however, was sufficient to initiate a significant rescue of *pth2* transcript levels in previously isolated zebrafish. *Pth2* transcription exhibited bi-directional dynamics: following the acute isolation of socially-reared fish, a rapid reduction in *pth2* levels was observed. Curiously, *pth2* expression tracked not just the presence of others, but also their density. The sensory modality that controls *pth2* expression was neither visual nor chemosensory in origin, but rather mechanical - induced by the movements of neighboring fish. Chemical ablation of the mechanosensitive neuromasts within the fish’s lateral line prevented the social-environment-induced rescue of *pth2* levels. In addition, mechanical perturbation of the water at frequencies similar to zebrafish tail movements was sufficient to rescue *pth2* levels in previously isolated fish. These data indicate a previously unappreciated role for a relatively unexplored neuropeptide, *pth2*, in both tracking and responding to the population density of an animal’s social environment. 

## Supplementary Code
This notebook contains the $\Delta C_{t}$ values from qPCR experiments and subsequent statistical analysis used in our above mentioned publication. All $\Delta C_{t}$ values are additionally reported in our supplementary tables.

In [168]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, ttest_ind
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy import stats
from scipy.stats import shapiro

# Fig. 1 - Transcriptional responses to social isolation
$\Delta$C$_{t}$ values as reported in supplementary table S2.
## 1 c) Validation of NGS data by qPCR at different developmental stages

In [49]:
val_dict = {'5 dpf': {'isolated': [5.491, 7.198, 3.960, 4.200, 5.292, 5.319, 4.565],
              'social': [3.932, 4.461, 1.164, 1.516, 3.230, 2.847, 2.630]},
    '8 dpf': {'isolated': [5.138, 6.305, 6.955, 6.689, np.nan, np.nan, np.nan],
             'social': [3.830, 5.197, 6.444, 5.291, np.nan, np.nan, np.nan]},
    '21 dpf': {'isolated': [4.383, 5.315, 5.601, 5.527, 4.784, 4.924, np.nan], 
              'social': [2.922, 3.733, 4.370, 4.000, 3.305, 3.385, np.nan]}}

df = pd.DataFrame.from_dict({(i,j): val_dict[i][j] 
                           for i in val_dict.keys() 
                           for j in val_dict[i].keys()})

[t, p] = ttest_rel(df['5 dpf']['isolated'], df['5 dpf']['social'])
print('5 dpf: t = ' + str(t) + ', p = ' + str(p))

[t, p] = ttest_rel(df['8 dpf']['isolated'].dropna(), df['8 dpf']['social'].dropna())
print('8 dpf: t = ' + str(t) + ', p = ' + str(p))

[t, p] = ttest_rel(df['21 dpf']['isolated'].dropna(), df['21 dpf']['social'].dropna())
print('21 dpf: t = ' + str(t) + ', p = ' + str(p))

5 dpf: t = 12.94446281825876, p = 1.308198194152707e-05
8 dpf: t = 5.419551280102047, p = 0.01232411256966652
21 dpf: t = 28.856000714011255, p = 9.365934158349621e-07


## 1 d) Expression differences of members of the *pth* gene family in response to different social environments

In [2]:
d = {'pth2 - isolated': [4.916,5.201,5.522,4.418,5.311,5.034,4.299,4.158,4.587,5.220], 
     'pth2 - social': [2.990,2.820,2.666,3.706,3.723,3.073,2.776,2.193,3.126,3.664],
    'pth1a - isolated': [4.107, 4.519, 4.336, 4.011, 2.352, 3.044, 3.789, 3.539, 4.733, 3.425],
    'pth1a - social': [4.877, 5.046, 4.854, 4.929, 3.544, 3.593, 4.289, 4.741, 3.431, 4.276],
    'pth1b - isolated': [-0.187, -0.143,-0.287, 1.146, 2.291,1.376, -0.089, -1.040, 0.494, 0.235],
    'pth1b - social': [-0.092,-0.296,-0.241,0.915,1.929,0.974,-0.581,-1.051,0.404,0.576],
    'pthla - isolated': [4.859,4.011,5.141,4.041,4.971,5.173,3.981,3.535,4.198,3.924],
    'pthla - social': [5.328,4.516,4.623,4.944,4.787,3.782,3.949,4.186,4.125,4.750],
    'pthlb - isolated': [8.514,8.067,8.542,7.680,10.390,8.702,7.936,7.869,8.173,8.308],
    'pthlb - social': [10.337,8.210,8.310,8.246,9.640,8.542,7.378,8.113,8.682,8.838],
    'pth4 - isolated': [4.892,4.082,4.644,3.761,5.194,5.129,4.172,3.591,4.519,4.379],
    'pth4 - social': [5.484,4.328,3.802,4.069,5.151,3.815,3.328,3.369,4.384,5.456]}
df = pd.DataFrame(data=d)
df.head()

,pth2 - isolated,pth2 - social,pth1a - isolated,pth1a - social,pth1b - isolated,pth1b - social,pthla - isolated,pthla - social,pthlb - isolated,pthlb - social,pth4 - isolated,pth4 - social
0,4.916,2.990,4.107,4.877,-0.187,-0.092,4.859,5.328,8.514,10.337,4.892,5.484
1,5.201,2.820,4.519,5.046,-0.143,-0.296,4.011,4.516,8.067,8.210,4.082,4.328
2,5.522,2.666,4.336,4.854,-0.287,-0.241,5.141,4.623,8.542,8.310,4.644,3.802
3,4.418,3.706,4.011,4.929,1.146,0.915,4.041,4.944,7.680,8.246,3.761,4.069
4,5.311,3.723,2.352,3.544,2.291,1.929,4.971,4.787,10.390,9.640,5.194,5.151


In [3]:
pvalues = []
[t, p] = ttest_rel(df['pth2 - isolated'], df['pth2 - social'])
print('For pth2, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)
[t, p] = ttest_rel(df['pth1a - isolated'], df['pth1a - social'])
print('For pth1a, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)
[t, p] = ttest_rel(df['pth1b - isolated'], df['pth1b - social'])
print('For pth1b, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)
[t, p] = ttest_rel(df['pthla - isolated'], df['pthla - social'])
print('For pthla, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)
[t, p] = ttest_rel(df['pthlb - isolated'], df['pthlb - social'])
print('For pthlb, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)
[t, p] = ttest_rel(df['pth4 - isolated'], df['pth4 - social'])
print('For pth4, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

For pth2, t = 9.818336030819468 and p = 2.083631937614688e-06
For pth1a, t = -2.54882884451511 and p = 0.015627371727670564
For pth1b, t = 1.5567921077943054 and p = 0.07697269970754803
For pthla, t = -0.5166201392269912 and p = 0.3089375633869814
For pthlb, t = -0.9187342265317263 and p = 0.19109856270501852
For pth4, t = 0.5126285172738125 and p = 0.3102773737945278


(array([ True, False, False, False, False, False]),
 array([2.50035833e-05, 9.37642304e-02, 3.07890799e-01, 6.20554748e-01,
        5.73295688e-01, 6.20554748e-01]),
 0.008512444610847103,
 0.008333333333333333)

We used this dataset to additionally test for normality using the Shapiro-Wilk test. Normality was accordingly assumed later on.

In [4]:
# normality test
stat, p = shapiro(df['pth2 - isolated'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = shapiro(df['pth2 - social'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=0.936, p=0.509
Statistics=0.926, p=0.409


# Fig. 2 - Transcriptional dynamics of *pth2*
## 2 a) Time dynamics: Increase of *pth2* levels correlate with time of exposure to conspecifics

In [5]:
d = {'isolated': [6.103,5.369,5.428,4.213,4.649,5.294,4.485,7.203,4.690,5.071,6.337,4.297,4.777,4.984,5.292,5.319,4.565],
     '30 min': [5.636, 4.858,5.322,3.889,4.119,4.369,5.259,5.667,4.652,4.157,4.475,4.580,4.716,4.610,4.599,np.nan, np.nan],
     '60 min': [5.637,4.997,3.957,4.148,4.006,4.039,4.107,4.652,4.481,4.887, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
     '180 min': [2.922,3.517,3.619,3.263,5.019,3.503,4.202,4.049,3.631,3.979,np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
     '12 hours': [4.140,3.287,3.104,2.511,2.735,2.622,2.938,2.653, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
     'social': [4.500,3.336,3.103,2.320,2.793,2.720,2.055,4.513,2.857,3.014,2.036,2.253,3.148,2.648,3.230,2.847,2.630]
}

df = pd.DataFrame(data=d)
df.head()

,isolated,30 min,60 min,180 min,12 hours,social
0,6.103,5.636,5.637,2.922,4.140,4.500
1,5.369,4.858,4.997,3.517,3.287,3.336
2,5.428,5.322,3.957,3.619,3.104,3.103
3,4.213,3.889,4.148,3.263,2.511,2.320
4,4.649,4.119,4.006,5.019,2.735,2.793


In [6]:
pvalues = []
[t, p] = ttest_ind(df['isolated'].dropna(),df['30 min'].dropna())
print('For 30 min, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_ind(df['isolated'].dropna(),df['60 min'].dropna())
print('For 60 min, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_ind(df['isolated'].dropna(),df['180 min'].dropna())
print('For 180 min, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_ind(df['isolated'].dropna(),df['12 hours'].dropna())
print('For 12 hours, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_ind(df['isolated'].dropna(),df['social'].dropna())
print('For social conditions, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_tsbh', is_sorted=False, returnsorted=False)

For 30 min, t = 1.8922402114296208 and p = 0.0340726057789864
For 60 min, t = 2.45013516874614 and p = 0.010812224586491411
For 180 min, t = 4.9546423012616 and p = 2.0956611347314803e-05
For 12 hours, t = 7.134543928544459 and p = 1.4423897122561982e-07
For social conditions, t = 8.783439730638207 and p = 2.4503498436730134e-10


(array([ True,  True,  True,  True,  True]),
 array([1.36290423e-02, 5.40611229e-03, 1.39710742e-05, 1.44238971e-07,
        4.90069969e-10]),
 0.010206218313011495,
 0.01)

## 2 b) Decrease of *pth2* levels after short term social isolation

In [7]:
d = {'isolated': [5.370,5.604,5.900,4.581,4.656,5.780,4.083,4.869,3.719,4.497,5.167,4.468,4.857],
    '3 hours': [3.182,3.507,3.086,3.250,3.969,3.511,2.799,2.447,2.233, np.nan, np.nan, np.nan, np.nan],
    '6 hours': [4.074,4.413,4.252,np.nan, np.nan, np.nan, np.nan, np.nan,np.nan, np.nan, np.nan, np.nan, np.nan],
    'social': [3.252,3.107,3.074,2.852,3.340,3.922,2.535,1.707,2.117,1.916,2.953,3.197, np.nan]}
df = pd.DataFrame(data = d)
df.head()

,isolated,3 hours,6 hours,social
0,5.370,3.182,4.074,3.252
1,5.604,3.507,4.413,3.107
2,5.900,3.086,4.252,3.074
3,4.581,3.250,NaN,2.852
4,4.656,3.969,NaN,3.340


In [8]:
pvalues = []
[t, p] = ttest_ind(df['social'].dropna(),df['3 hours'].dropna())
print('For 3 hours, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_ind(df['social'].dropna(),df['6 hours'].dropna())
print('For 6 hours, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

For 3 hours, t = -1.0401920159552156 and p = 0.15565361352597956
For 6 hours, t = -3.659605880548592 and p = 0.0014419732748514522


(array([False,  True]),
 array([0.31130723, 0.00576789]),
 0.025320565519103666,
 0.025)

## 2 c) *pth2* dynamics at the juvenile stage

In [9]:
d = {'social': [3.733,4.370,4.000,3.305,3.385],
    '24 hours': [5.594, 5.834,5.647,4.703,4.384],
    'isolated': [5.315,5.601,5.527,4.784,4.924]}

df = pd.DataFrame(data=d)
df.head()

,social,24 hours,isolated
0,3.733,5.594,5.315
1,4.370,5.834,5.601
2,4.000,5.647,5.527
3,3.305,4.703,4.784
4,3.385,4.384,4.924


In [10]:
pvalues = []
[t, p] = ttest_rel(df['social'].dropna(),df['isolated'].dropna())
print('For social conditions, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_rel(df['isolated'].dropna(),df['24 hours'].dropna())
print('For 24 hours isolation, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

For social conditions, t = -23.603345885760536 and p = 9.550978099311588e-06
For 24 hours isolation, t = -0.014754358310279174 and p = 0.4944673665483954


(array([ True, False]),
 array([3.82039124e-05, 9.88934733e-01]),
 0.025320565519103666,
 0.025)

## 2 d) *pth2* dynamics in adult zebrafish


In [11]:
d = {'isolated': [2.474,1.713,1.999, np.nan],
    'social': [0.727,0.302,0.816,0.932],
    'short social': [0.363,0.854,0.985,0.540],
    'short iso': [3.489,3.591,3.192,3.426]}

df = pd.DataFrame(data=d)
df.head()

,isolated,social,short social,short iso
0,2.474,0.727,0.363,3.489
1,1.713,0.302,0.854,3.591
2,1.999,0.816,0.985,3.192
3,NaN,0.932,0.540,3.426


In [12]:
pvalues = []

[t, p] = ttest_ind(df['short social'].dropna(),df['isolated'].dropna())
print('comparing short social and isolated, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p/2)

[t, p] = ttest_ind(df['short social'].dropna(),df['social'].dropna())
print('comparing short social and social, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p/2)

[t, p] = ttest_ind(df['short iso'].dropna(),df['social'].dropna())
print('comparing short isolation and social, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p/2)

[t, p] = ttest_ind(df['short iso'].dropna(),df['isolated'].dropna())
print('comparding short isolation and isolated, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p/2)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

comparing short social and isolated, t = -5.490297510789816 and p = 0.0013681014836495308
comparing short social and social, t = -0.04423416862896142 and p = 0.48307657234585033
comparing short isolation and social, t = 16.926277706574503 and p = 1.3591597835397753e-06
comparding short isolation and isolated, t = 6.458645684578994 and p = 0.0006622502007027491


(array([ True, False,  True,  True]),
 array([1.82413531e-03, 4.83076572e-01, 5.43663913e-06, 1.32450040e-03]),
 0.012741455098566168,
 0.0125)

# Fig. 3 - Sensory perception of conspecifics
## 3 b) chemosensory rescue attempt

In [13]:
d = {'isolated': [5.862,5.877,5.645],
    'medium swap': [5.522,6.056,5.910],
    'social': [3.284, 3.414, 2.703]} 

df = pd.DataFrame(data=d)
df.head()

,isolated,medium swap,social
0,5.862,5.522,3.284
1,5.877,6.056,3.414
2,5.645,5.910,2.703


In [14]:
pvalues = []
[t, p] = ttest_rel(df['social'].dropna(),df['isolated'].dropna())
print('comparing social and isolated, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_rel(df['medium swap'].dropna(),df['isolated'].dropna())
print('comparing isolated and chemosensory rescue, t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

comparing social and isolated, t = -18.431967920580387 and p = 0.0014652585395614993
comparing isolated and chemosensory rescue, t = 0.18344947948338597 and p = 0.4356797111112813


(array([ True, False]),
 array([0.00586103, 0.87135942]),
 0.025320565519103666,
 0.025)

## 3 d) Visual Rescue Attempt

In [16]:
d = {'age': [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
            21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21],
    'visual': [0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,
              0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1],
    'physical': [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,
                0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1],
    'value': [5.531,6.070,5.900,6.244,6.190,
              4.932,5.204,5.539,6.016,4.981,
              3.786,4.548,4.098,4.077,4.993,
              3.802,4.790,3.887,4.371,4.576,
              5.655,5.718,6.042,5.552,5.661,5.583,
              5.541,6.621,5.727,5.095,5.031,5.454,
              4.702,4.778,4.762,4.498,4.296,4.998,
              5.114,4.937,5.330,4.060,4.915,4.448]}

df = pd.DataFrame(data = d)

In [39]:
# 5 dpf
df5dpf = df[df['age']==5]

formula = 'value ~ visual + physical + visual:physical'
model = ols(formula, df5dpf).fit()
aov_table = anova_lm(model, typ=2)
print('5 dpf - ANOVA results')
print(aov_table.round(6))

# 21 dpf
df21dpf = df[df['age']==21]

formula = 'value ~ visual + physical + visual:physical'
model = ols(formula, df21dpf).fit()
aov_table = anova_lm(model, typ=2)
print('21 dpf - ANOVA results')
print(aov_table.round(6))


5 dpf - ANOVA results
                   sum_sq    df          F    PR(>F)
visual           0.557446   1.0   3.213884  0.091941
physical         9.355752   1.0  53.939397  0.000002
visual:physical  0.507848   1.0   2.927936  0.106372
Residual         2.775189  16.0        NaN       NaN
21 dpf - ANOVA results
                   sum_sq    df          F    PR(>F)
visual           0.000033   1.0   0.000204  0.988736
physical         4.897873   1.0  30.641831  0.000020
visual:physical  0.095256   1.0   0.595936  0.449159
Residual         3.196854  20.0        NaN       NaN


Since there is apparently no interaction effect, no post-hoc tests need to be performed. 

## 3 e) Comparison of *pth2* levels after social rescue under light and in darkness.

In [17]:
d = {'isolated':[5.336,5.718,5.740,6.304,6.211,5.929],
    'light':[3.402,4.056,4.867,4.710,4.897,4.276],
    'dark':[4.261,5.221,4.624,4.879,4.655,4.567]}

df = pd.DataFrame(data=d)
df.head()

,isolated,light,dark
0,5.336,3.402,4.261
1,5.718,4.056,5.221
2,5.740,4.867,4.624
3,6.304,4.710,4.879
4,6.211,4.897,4.655


In [18]:
pvalues = []

[t, p] = ttest_rel(df['isolated'].dropna(),df['light'].dropna())
print('comparing isolated and social (light), t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_rel(df['isolated'].dropna(),df['dark'].dropna())
print('comparing isolated and social (dark), t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

[t, p] = ttest_rel(df['light'].dropna(),df['dark'].dropna())
print('comparing social (light) and social (dark), t = '+ str(t) + ' and p = ' + str(p/2))
pvalues.append(p)

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

comparing isolated and social (light), t = 10.037646730561294 and p = 8.39471166533326e-05
comparing isolated and social (dark), t = 7.58665609686374 and p = 0.0003157879102815708
comparing social (light) and social (dark), t = -1.4163586870605946 and p = 0.10792224230982565


(array([ True,  True, False]),
 array([0.00050368, 0.00094736, 0.21584448]),
 0.016952427508441503,
 0.016666666666666666)

# Fig. 4 - Specific mechanosensation triggers transcription of *pth2*
## 4 b) Impact of lateral line ablation on *pth2* levels

In [19]:
d = {'E3_iso': [8.132,7.327,6.928,3.923,5.438,4.720,4.787,4.837,5.123],
    'E3_soc': [5.769,5.128,4.251,3.243,3.635,3.861,3.525,3.979,3.538],
    'CuSO4_iso': [6.472,6.925,5.194,4.593,6.217, np.nan, np.nan, np.nan, np.nan],
    'CuSO4_soc': [6.859,6.784,4.323,4.439,5.093, np.nan, np.nan, np.nan, np.nan],
    'Neo_iso': [4.878,5.018,4.808,5.203,4.678, np.nan, np.nan, np.nan, np.nan],
    'Neo_soc': [4.545,5.229,4.620,4.365,4.640, np.nan, np.nan, np.nan, np.nan]}

df = pd.DataFrame(data=d)
df.head()

,E3_iso,E3_soc,CuSO4_iso,CuSO4_soc,Neo_iso,Neo_soc
0,8.132,5.769,6.472,6.859,4.878,4.545
1,7.327,5.128,6.925,6.784,5.018,5.229
2,6.928,4.251,5.194,4.323,4.808,4.620
3,3.923,3.243,4.593,4.439,5.203,4.365
4,5.438,3.635,6.217,5.093,4.678,4.640


In [20]:
pvalues = []

[t, p] = ttest_rel(df['E3_iso'].dropna(), df['E3_soc'].dropna())
pvalues.append(p)
print('For untreated animals, t = ' + str(t) + ' p = ' + str(p))

[t, p] = ttest_rel(df['CuSO4_iso'].dropna(), df['CuSO4_soc'].dropna())
pvalues.append(p)
print('For CuSO4-treated animals, t = ' + str(t) + ' p = ' + str(p))

[t, p] = ttest_rel(df['Neo_iso'].dropna(), df['Neo_soc'].dropna())
pvalues.append(p)
print('For Neomycin-treated animals, t = ' + str(t) + ' p = ' + str(p))

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

For untreated animals, t = 6.571262910391263 p = 0.00017454549429944867
For CuSO4-treated animals, t = 1.3938955189093376 p = 0.23579614130931398
For Neomycin-treated animals, t = 1.354739985557677 p = 0.24696415796455468


(array([ True, False, False]),
 array([0.00052364, 0.24696416, 0.24696416]),
 0.016952427508441503,
 0.016666666666666666)

## 4 c) Impact of other species' movement: presence of artemia

In [21]:
d = {'isolated': [5.569,5.559,5.426,6.269,6.556,6.335],
    'artemia14': [4.776,5.610,5.306, np.nan, np.nan, np.nan],
    'artemia1000': [np.nan, np.nan, np.nan,6.132, 6.543, 7.352],
    'social': [np.nan, np.nan, np.nan, 3.919, 4.667, 4.308]}

df = pd.DataFrame(data=d)
df.head()

,isolated,artemia14,artemia1000,social
0,5.569,4.776,NaN,NaN
1,5.559,5.610,NaN,NaN
2,5.426,5.306,NaN,NaN
3,6.269,NaN,6.132,3.919
4,6.556,NaN,6.543,4.667


In [22]:
pvalues = []

[t, p] = ttest_rel(df['isolated'].iloc[0:3], df['artemia14'].iloc[0:3])
pvalues.append(p)
print('Addition of 14 artemia results in t = ' + str(t) + ' and p = ' + str(p))

[t, p] = ttest_rel(df['isolated'].iloc[3:7], df['artemia1000'].iloc[3:7])
pvalues.append(p)
print('Addition of 1000 artemia results in t = ' + str(t) + ' and p = ' + str(p))

[t, p] = ttest_rel(df['isolated'].iloc[3:7], df['social'].iloc[3:7])
pvalues.append(p)
print('Addition of 14 zebrafish results in t = ' + str(t) + ' and p = ' + str(p))

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

Addition of 14 artemia results in t = 1.1153934655503712 and p = 0.38072890915484936
Addition of 1000 artemia results in t = -0.7901445982484151 and p = 0.5122497752889774
Addition of 14 zebrafish results in t = 15.289886630578483 and p = 0.00425026281297711


(array([False, False,  True]),
 array([0.51224978, 0.51224978, 0.01275079]),
 0.016952427508441503,
 0.016666666666666666)

## 4 e) Impact of mechanical stimulation

In [23]:
d = {'isolated': [4.872,5.272,4.589,4.506,4.941,
                  4.589,4.506,4.941,5.019,4.847,
                  6.272,4.884,4.769,6.565,4.990,
                  4.309,4.423,4.206,4.895,4.733,
                  5.263,5.169,4.634,4.901,5.572,
                  4.614,5.017,4.067],
    'periodic': [4.289,4.944,4.891,4.522,4.706,
                 4.891,4.522,4.706,np.nan,np.nan,
                 np.nan, np.nan, np.nan, np.nan, np.nan,
                 np.nan, np.nan, np.nan, np.nan, np.nan,
                 np.nan, np.nan, np.nan, np.nan, np.nan,
                 np.nan, np.nan, np.nan],
    '3 hours': [np.nan, np.nan, np.nan, np.nan, np.nan,
                np.nan, np.nan, np.nan, 4.263, 4.037,
                5.486,4.631,4.944,5.731,4.623,
                4.432,3.764,3.851,np.nan, np.nan, 
                np.nan, np.nan, np.nan, np.nan, np.nan,
                np.nan, np.nan, np.nan],
    '6 hours': [np.nan, np.nan, np.nan, np.nan, np.nan,
                np.nan, np.nan, np.nan, np.nan, np.nan,
                np.nan, np.nan, np.nan, np.nan, np.nan,
                np.nan, np.nan, np.nan, 4.217,4.266,
                4.185,4.293,4.423,4.254,4.188,
                4.395,3.666,3.623]}

df = pd.DataFrame(data=d)

In [24]:
pvalues = []

[t, p] = ttest_rel(df['isolated'].iloc[0:8], df['periodic'].iloc[0:8])
pvalues.append(p)
print('For periodic stimulation, t = ' + str(t) + ' and p = ' + str(p))

[t, p] = ttest_rel(df['isolated'].iloc[8:18], df['3 hours'].iloc[8:18])
pvalues.append(p)
print('For 3 hours of random stimulation, t = ' + str(t) + ' and p = ' + str(p))

[t, p] = ttest_rel(df['isolated'].iloc[18:28], df['6 hours'].iloc[18:28])
pvalues.append(p)
print('For 6 hours of random stimulation, t = ' + str(t) + ' and p = ' + str(p))

multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

For periodic stimulation, t = 0.8504497340075354 and p = 0.42320753025009483
For 3 hours of random stimulation, t = 3.761233161640674 and p = 0.004476493708684494
For 6 hours of random stimulation, t = 5.437773976058705 and p = 0.00041217482919133643


(array([False,  True,  True]),
 array([0.42320753, 0.00671474, 0.00123652]),
 0.016952427508441503,
 0.016666666666666666)

In [25]:
ttest_ind(df['6 hours'].iloc[18:28],df['3 hours'].iloc[8:18])

Ttest_indResult(statistic=-1.8817136316265357, pvalue=0.07614800560969336)

# Supplemental figures


## Fig. S1 b) Response of the stress axis to social exposure

In [26]:
Ct_dict = {'StAR': {'isolated': [1.76,3.55,4.09,1.08,1.29,1.53], 
                      'social': [2.99,1.75,1.96,2.17,1.39,1.76]},
             'nr3c1': {'isolated': [-0.95,-1.32,-1.00,-1.87,-2.22,-1.76], 
                       'social': [-1.07,-1.12,-0.97,-1.95,-1.60,-1.70]}}


df = pd.DataFrame.from_dict({(i,j): Ct_dict[i][j] 
                           for i in Ct_dict.keys() 
                           for j in Ct_dict[i].keys()})


[t,p] = ttest_ind(df['StAR']['isolated'], df['StAR']['social'])
print('Comparing StAR expression levels between animals reared in isolation and socially exposed (3 hours), results in t = '
     + str(t) + ' and p = ' + str(p))
[t,p] = ttest_ind(df['nr3c1']['isolated'], df['nr3c1']['social'])
print('Comparing nr3c1 expression levels between animals reared in isolation and socially exposed (3 hours), results in t = '
     + str(t) + ' and p = ' + str(p))

Comparing StAR expression levels between animals reared in isolation and socially exposed (3 hours), results in t = 0.3767183391990697 and p = 0.7142532890330876
Comparing nr3c1 expression levels between animals reared in isolation and socially exposed (3 hours), results in t = -0.44618803711471194 and p = 0.6649673964527298


## Fig. S5: Impact of sex and density on *pth2* expression in adult zebrafish

Most of our results were obtained from larval or juvenile zebrafish. At these stages, sexual differentiation has not taken place yet. We adressed the question whether sex is a factor in *pth2* expression strength at later, adult stages of development.

## S5 a) Impact of sex on *pth2* expression in adult zebrafish

In [27]:
d = {'male': [0.157,0.254,-0.419,-0.329],
    'female': [0.093,-0.035,-0.280,-0.453]}

df = pd.DataFrame(data=d)
df.head()

,male,female
0,0.157,0.093
1,0.254,-0.035
2,-0.419,-0.280
3,-0.329,-0.453


In [28]:
[t, p] = ttest_ind(df['male'], df['female'])
print('Comparing pth2 levels of adult males and females, t = ' + str(t) + ' and p = ' + str(p))

Comparing pth2 levels of adult males and females, t = 0.4043141263942289 and p = 0.6999994670017485


## S5 b) Impact of density on *pth2* expression in adult zebrafish

In [41]:
d = {'5': [1.354,1.926,2.082,1.916,2.387,1.790],
    '35': [1.094,1.277,1.377,1.972,1.618,1.376]}

df = pd.DataFrame(data=d)
df.head()

,5,35
0,1.354,1.094
1,1.926,1.277
2,2.082,1.377
3,1.916,1.972
4,2.387,1.618


In [42]:
[t, p] = ttest_ind(df['5'], df['35'])
print('Comparing pth2 levels in adults of different densities, results in t = ' + str(t) + ' and p = ' + str(p))

Comparing pth2 levels in adults of different densities, results in t = 2.442944835707187 and p = 0.034673841476036096


## Fig. S6 a) Impact of the rearing condition on locomotion

In [ ]:
rearing_dict = {'isolation': [1.46, 0.67, 1.24, 1.12, 1.33, 1.17, 1.13, 1.16, 0.58, 1.63, 1.41, 1.13, 1.33, 1.12],
               'social': [1.22, 2.22, 0.79, 0.57, 0.84, 1.07, 0.92, 1.10, 1.37, 1.40, 1.01, np.nan, np.nan, np.nan]}

df = pd.DataFrame(data=rearing_dict)

[t, p] = ttest_ind(df['isolation'], df['social'].iloc[0:-3])
print('Comparing average velocity of animals reared in isolation or with conspecifics, t = ' + str(t) + ' and p = ' + str(p))

## Fig. S6 b) Impact of the presence of conspecifics on locomotion

In [ ]:
presence_dict = {'isolation': [0.70, 0.33, 0.42, 0.78, 0.70, 0.82, 0.85, 0.59],
               'social': [ 0.60, 0.69, 0.62, 0.75, 0.43, 0.58, 0.64, 0.59]}

df = pd.DataFrame(data=presence_dict)

[t, p] = ttest_ind(df['isolation'], df['social'].iloc[0:-3])
print('Comparing average velocity of isolation-reared animals in the presence or absence of conspecifics, t = ' + str(t) + ' and p = ' + str(p))

## Fig. S6 d) Impact of visual access to conspecifics on locomotion

In [ ]:



df = {'age': [7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,
              7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,
              7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7, 
              7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7, 
              14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,
              14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,
              14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,
              14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,
              21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,
              21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,
              21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,
              21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,
              28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,
              28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,
              28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,
              28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
             ],
      'rearing': [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      'visual_access': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                        1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
      'values':[2.9, 4.9, 3.5, 4.9, np.nan, 3.4, 1.4, 3.5, 4.4, 4.0, 3.1, np.nan, 2.2, 3.5, 5.0, 3.8, np.nan, 5.1,
                2.8, 2.4, 2.9, 2.6, 3.4, 2.6, np.nan, 3.1, 2.3, 3.6, 2.5, 2.1, 2.2, 1.3, np.nan, 0.9, 4.1, 4.3,
                2.3, 1.9, 1.7, 1.5, 1.9, 2.2, 1.8, 4.4, 2.8, 1.8, np.nan, 2.6, 1.5, 2.3, 2.5, 2.1, 1.7, 2.8,
                2.3, 3.1, 1.7, 1.8, 2.1, 1.7, 2.5, 3.3, 2.5, 1.5, 1.8, 2.5, 2.3, 1.4, 2.1, 1.4, 1.8, 1.9,
                
                7.3, np.nan, 6.7, 10.6, 9.1, 2.5, 4.8, 5.8, 6.0, 7.4, 8.1, 3.5, 5.4, 3.8, 5.0, 8.0, 0.5, 0.9,
                4.0, 12.7, 11.6, 12.7, 10.0, 2.3, 2.4, 5.8, 3.8, 7.3, 5.6, 4.4, 4.4, 3.4, 4.0, 8.6, 1.3, 1.0,
                2.8, 2.9, 4.3, 0.3, 6.6, 8.0, 6.6, 1.2, 5.6, np.nan, 1.5, np.nan, 2.8, 3.1, 10.4, 4.3, 0.4, np.nan,
                3.8, 4.1, 6.8, 1.3, 5.7, 5.4, 5.9, 3.5, 2.5, np.nan, 5.1, np.nan, 2.2, 4.4, 2.0, 1.8, np.nan, np.nan,
               
                10.6, 7.9, 13.0, 9.0, 10.1, 6.5, 8.1, 9.0, 11.0, 10.9, 12.2, 9.3, 14.2, 10.9, 13.9, 8.4, 10.7, 5.8,
                8.4, 10.3, 8.6, 7.4, 8.4, 10.0, 7.3, np.nan, 7.7, 9.3, 13.1, 9.0, 12.8, 7.5, np.nan, 7.7, 7.1, 4.5,
                8.9, 6.5, 7.2, 8.6, 7.5, 5.3, 8.0, 10.7, 7.6, 6.5, 7.7, 8.7, 8.0, 5.1, 7.3, 8.2, 10.3, np.nan,
                9.0, 8.5, np.nan, np.nan, 11.9, 6.6, 6.8, 8.8, 6.8, 7.6, 7.4, 10.3, np.nan, 9.1, 8.5, np.nan, 12.3, np.nan,
                
                13.6, 15.5, 4.5, 12.8, 5.0, 6.9, 7.5, 13.3, 13.4, 17.5, np.nan, 13.0, np.nan, np.nan, 8.2, 8.8, 16.2, 18.4,                
                19.2, 17.1, 13.7, 18.6, 8.9, 12.3, 21.5, 21.6, 14.3, 20.6, 9.8, 19.9, 3.1, 7.4, 18.9, 10.1, 11.6, 21.9,
                12.8, np.nan, 3.1, 19.8, 2.9, 11.0, 16.3, 13.1, 11.4, 10.1, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                9.6, np.nan, 2.6, 15.5, 3.4, 12.4, 13.5, 11.5, 13.7, 8.9, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]}
                        


df = pd.DataFrame(data = df)

formula = 'values ~ age + rearing + visual_access + age:rearing + age:visual_access + rearing:visual_access + age:rearing:visual_access'


model = ols(formula, df).fit()
aov_table = anova_lm(model, typ=2)

print(aov_table.round(6))


In [ ]:
va_dict = {'7 dpf': {'rearing': {'social': {'no va':[2.9, 4.9, 3.5, 4.9, np.nan, 3.4, 1.4, 3.5, 4.4, 4.0, 3.1, np.nan, 2.2, 3.5, 5.0, 3.8, np.nan, 5.1],
                                            'va':[2.8, 2.4, 2.9, 2.6, 3.4, 2.6, np.nan, 3.1, 2.3, 3.6, 2.5, 2.1, 2.2, 1.3, np.nan, 0.9, 4.1, 4.3]},
                               'isolated': {'no va': [2.3, 1.9, 1.7, 1.5, 1.9, 2.2, 1.8, 4.4, 2.8, 1.8, np.nan, 2.6, 1.5, 2.3, 2.5, 2.1, 1.7, 2.8],
                                         'va': [2.3, 3.1, 1.7, 1.8, 2.1, 1.7, 2.5, 3.3, 2.5, 1.5, 1.8, 2.5, 2.3, 1.4, 2.1, 1.4, 1.8, 1.9]}}},
           '14 dpf':{'rearing': {'social':{'no va': [7.3, np.nan, 6.7, 10.6, 9.1, 2.5, 4.8, 5.8, 6.0, 7.4, 8.1, 3.5, 5.4, 3.8, 5.0, 8.0, 0.5, 0.9],
                                         'va': [4.0, 12.7, 11.6, 12.7, 10.0, 2.3, 2.4, 5.8, 3.8, 7.3, 5.6, 4.4, 4.4, 3.4, 4.0, 8.6, 1.3, 1.0]},
                               'isolated':{'no va': [2.8, 2.9, 4.3, 0.3, 6.6, 8.0, 6.6, 1.2, 5.6, np.nan, 1.5, np.nan, 2.8, 3.1, 10.4, 4.3, 0.4, np.nan],
                                         'va': [3.8, 4.1, 6.8, 1.3, 5.7, 5.4, 5.9, 3.5, 2.5, np.nan, 5.1, np.nan, 2.2, 4.4, 2.0, 1.8, np.nan, np.nan]}}},
           '21 dpf':{'rearing': {'social':{'no va': [10.6, 7.9, 13.0, 9.0, 10.1, 6.5, 8.1, 9.0, 11.0, 10.9, 12.2, 9.3, 14.2, 10.9, 13.9, 8.4, 10.7, 5.8],
                                         'va': [8.4, 10.3, 8.6, 7.4, 8.4, 10.0, 7.3, np.nan, 7.7, 9.3, 13.1, 9.0, 12.8, 7.5, np.nan, 7.7, 7.1, 4.5]},
                               'isolated':{'no va': [8.9, 6.5, 7.2, 8.6, 7.5, 5.3, 8.0, 10.7, 7.6, 6.5, 7.7, 8.7, 8.0, 5.1, 7.3, 8.2, 10.3, np.nan],
                                         'va': [9.0, 8.5, np.nan, np.nan, 11.9, 6.6, 6.8, 8.8, 6.8, 7.6, 7.4, 10.3, np.nan, 9.1, 8.5, np.nan, 12.3, np.nan]}}},
           '28 dpf':{'rearing': {'social':{'no va': [13.6, 15.5, 4.5, 12.8, 5.0, 6.9, 7.5, 13.3, 13.4, 17.5, np.nan, 13.0, np.nan, np.nan, 8.2, 8.8, 16.2, 18.4],
                                         'va': [19.2, 17.1, 13.7, 18.6, 8.9, 12.3, 21.5, 21.6, 14.3, 20.6, 9.8, 19.9, 3.1, 7.4, 18.9, 10.1, 11.6, 21.9]},
                               'isolated':{'no va': [12.8, np.nan, 3.1, 19.8, 2.9, 11.0, 16.3, 13.1, 11.4, 10.1, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,],
                                         'va': [9.6, np.nan, 2.6, 15.5, 3.4, 12.4, 13.5, 11.5, 13.7, 8.9, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]}}}}
 
    
df = pd.DataFrame.from_dict({(i,j, k, l): va_dict[i][j][k][l] 
                           for i in va_dict.keys() 
                           for j in va_dict[i].keys()
                             for k in va_dict[i][j].keys()
                             for l in va_dict[i][j][k].keys()
                            })

ages = ['7 dpf', '14 dpf', '21 dpf', '28 dpf']
pvalues = []
for age in ages:
    [t, p] = ttest_ind(df[age]['rearing']['isolated'].dropna(), df[age]['rearing']['social'].dropna())
    print(age + ': t = ' + str(t) + ' and p = ' + str(p))
    pvalues.append(p[0])
    pvalues.append(p[1])
    
pvalues
multipletests(pvalues, alpha=0.05, method='holm', is_sorted=False, returnsorted=False)

## Fig. S7 - Artificial mechanical stimulation
As we do not show all experiments we performed with artificial mechanical stimulation (piezo-paradigm) in the main figures, we provide a more comprehensive list in the supplements.
## S7 a) Additional stimulation paradigms and their effect on *pth2* expression

In [31]:
## 70 Hz - continuous stimulation
d = {'isolated70cont':[4.890,5.970,5.018,6.747,5.424,5.239],
     'piezo70cont':[8.0276,6.408,6.375,5.259,5.115,4.708],
    # 70 Hz - with kin-imbued water
     'isolated70contKIW':[4.386,5.163, np.nan, np.nan, np.nan, np.nan],
     'piezo70contKIW':[4.347,5.289, np.nan, np.nan, np.nan, np.nan],
    # 70 Hz, two piezo pieces, 300 msec
     'isolated70_2_300': [4.589,4.506,4.941, np.nan, np.nan, np.nan],
     'piezo70_2_300': [4.891,4.522,4.706, np.nan, np.nan, np.nan],
    # 70 Hz, two piezo pieces, 5000 msec
     'isolated70_2_5000': [4.769,4.300, np.nan, np.nan, np.nan, np.nan],
     'piezo70_2_5000': [4.426,4.620, np.nan, np.nan, np.nan, np.nan],
    # 70 Hz, two piezo pieces + rubber, 300 msec
     'isolated70_2r_300': [4.523,4.075,4.334, np.nan, np.nan, np.nan],
     'piezo70_2r_300': [4.412,4.508,4.346, np.nan, np.nan, np.nan],
    # 70 Hz, two piezo pieces + rubber, 500 msec
     'isolated70_2r_500': [3.847,4.284, np.nan, np.nan, np.nan, np.nan],
     'piezo70_2r_500': [4.171,3.880, np.nan, np.nan, np.nan, np.nan], 
    # 70 Hz, two piezo pieces + rubber, 700 msec
     'isolated70_2r_700': [4.480,5.042,5.383, np.nan, np.nan, np.nan],
     'piezo70_2r_700': [5.900,5.419,4.924, np.nan, np.nan, np.nan],  
    # 70 Hz, two piezo pieces + rubber, 800 msec
     'isolated70_2r_800': [4.757,4.294,5.384,4.996,5.501, np.nan],
     'piezo70_2r_800': [4.510,4.438,4.893,7.979,5.773, np.nan]     
    }

df = pd.DataFrame(data=d)
df.head(10)

,isolated70cont,piezo70cont,isolated70contKIW,piezo70contKIW,isolated70_2_300,piezo70_2_300,isolated70_2_5000,piezo70_2_5000,isolated70_2r_300,piezo70_2r_300,isolated70_2r_500,piezo70_2r_500,isolated70_2r_700,piezo70_2r_700,isolated70_2r_800,piezo70_2r_800
0,4.890,8.0276,4.386,4.347,4.589,4.891,4.769,4.426,4.523,4.412,3.847,4.171,4.480,5.900,4.757,4.510
1,5.970,6.4080,5.163,5.289,4.506,4.522,4.300,4.620,4.075,4.508,4.284,3.880,5.042,5.419,4.294,4.438
2,5.018,6.3750,NaN,NaN,4.941,4.706,NaN,NaN,4.334,4.346,NaN,NaN,5.383,4.924,5.384,4.893
3,6.747,5.2590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996,7.979
4,5.424,5.1150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.501,5.773
5,5.239,4.7080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
pvalues = []
# 70 Hz, continuous stimulation
[t, p] = ttest_rel(df['isolated70cont'].dropna(), df['piezo70cont'].dropna())
pvalues.append(p)
print('for continued stimulation with 70 Hz, t = ' + str(t) + ' and p = ' + str(p))

# 70 Hz, continuous stimulation + kin-imbued water
[t, p] = ttest_rel(df['isolated70contKIW'].dropna(), df['piezo70contKIW'].dropna())
pvalues.append(p)
print('for continued stimulation with 70 Hz in the presence of kin-imbued water, t = ' + str(t) + ' and p = ' + str(p))

# 70 Hz, 300 msec stimulation with two-ended piezo
[t, p] = ttest_rel(df['isolated70_2_300'].dropna(), df['piezo70_2_300'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (300 msec interval) with a two-ended piezo, t = ' + str(t) + ' and p = ' + str(p))

# 70 Hz, 5000 msec stimulation with two-ended piezo
[t, p] = ttest_rel(df['isolated70_2_5000'].dropna(), df['piezo70_2_5000'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (5000 msec interval) with a two-ended piezo, t = ' + str(t) + ' and p = ' + str(p))

# 70 Hz, 300 msec stimulation with two-ended piezo + rubber
[t, p] = ttest_rel(df['isolated70_2r_300'].dropna(), df['piezo70_2r_300'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (300 msec interval) with a two-ended piezo and rubber extension, t = ' + str(t) + ' and p = ' + str(p))

# 70 Hz, 500 msec stimulation with two-ended piezo + rubber
[t, p] = ttest_rel(df['isolated70_2r_500'].dropna(), df['piezo70_2r_500'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (500 msec interval) with a two-ended piezo and rubber extension, t = ' + str(t) + ' and p = ' + str(p))


# 70 Hz, 700 msec stimulation with two-ended piezo + rubber
[t, p] = ttest_rel(df['isolated70_2r_700'].dropna(), df['piezo70_2r_700'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (700 msec interval) with a two-ended piezo and rubber extension, t = ' + str(t) + ' and p = ' + str(p))


# 70 Hz, 800 msec stimulation with two-ended piezo + rubber
[t, p] = ttest_rel(df['isolated70_2r_800'].dropna(), df['piezo70_2r_800'].dropna())
pvalues.append(p)
print('for stimulation with 70 Hz (800 msec interval) with a two-ended piezo and rubber extension, t = ' + str(t) + ' and p = ' + str(p))


# correcting for multiple testing
multipletests(pvalues, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

for continued stimulation with 70 Hz, t = -0.6510176314629132 and p = 0.5437643744670104
for continued stimulation with 70 Hz in the presence of kin-imbued water, t = -0.5272727272727284 and p = 0.6910949024451758
for stimulation with 70 Hz (300 msec interval) with a two-ended piezo, t = -0.1783470416587776 and p = 0.8748806117176366
for stimulation with 70 Hz (5000 msec interval) with a two-ended piezo, t = 0.034690799396681266 and p = 0.9779240041418579
for stimulation with 70 Hz (300 msec interval) with a two-ended piezo and rubber extension, t = -0.6759513415438768 and p = 0.5687582584700501
for stimulation with 70 Hz (500 msec interval) with a two-ended piezo and rubber extension, t = 0.10989010989010935 and p = 0.9303213616722098
for stimulation with 70 Hz (700 msec interval) with a two-ended piezo and rubber extension, t = -0.8205820588864406 and p = 0.49812714373933153
for stimulation with 70 Hz (800 msec interval) with a two-ended piezo and rubber extension, t = -0.84792348858

(array([False, False, False, False, False, False, False, False]),
 array([0.977924, 0.977924, 0.977924, 0.977924, 0.977924, 0.977924,
        0.977924, 0.977924]),
 0.006391150954545011,
 0.00625)

## Fig. S8 - impact of genotype on *pth2* transcription
As most of our experiments were conducted with the same genotype, we validated two of our key experiments with another strain of ours (*mitfa$^{-/-}$*). These were the regular social rescue experiment (social exposure of previously isolated fish for 3 hours) and the visual rescue experiment from fig. 3 c) and d). 

In [50]:
S6a = {'isolated':[5.491, 7.198, 3.960, 4.200, 5.568, 4.302, 5.828],
     'social':[3.932, 4.461, 1.164, 1.516, 3.699, 2.961, 4.516]}

S6b = {'visual': [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1],
    'physical': [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1],
    'value': [5.697, 5.460, 6.254, 6.221, 5.689, 6.025,
             5.243, 4.755, 5.597, 6.093, 5.428, 5.818,
             3.386, 3.208, 3.914, 4.375, 3.586, 3.586,
             3.704, 3.429, 3.436, 4.146, 3.632, 4.335]}

S6a = pd.DataFrame(data = S6a)
S6b = pd.DataFrame(data = S6b)

In [52]:
[t, p] = ttest_rel(S6a['isolated'], S6a['social'])
print('for mitfa-/- fish, there is a difference between socially reared and isolated animals with t = ' + str(t) + ' and p = ' + str(p))


for mitfa-/- fish, there is a difference between socially reared and isolated animals with t = 7.980665396844242 and p = 0.00020621437416382658


In [35]:
formula = 'value ~ visual + physical + visual:physical'
model = ols(formula, S6b).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table.round(8))

                    sum_sq    df           F    PR(>F)
visual            0.132759   1.0    0.833312  0.372190
physical         23.094702   1.0  144.962228  0.000000
visual:physical   0.384813   1.0    2.415420  0.135828
Residual          3.186306  20.0         NaN       NaN
